In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import nltk
import pandas
import re
import numpy as np
import string
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

In [2]:
dataframe = pandas.read_csv("Phishing_Email.csv")
trainemails, testemails = train_test_split(dataframe, test_size=0.2)

In [4]:
dataframe.head()

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\r\nHello I am your hot lil horny toy.\r\n ...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [5]:
trainemailstext = trainemails['Email Text'].values.tolist()   # '0' refers to the review text
trainemailstype = trainemails['Email Type'].values.tolist()   # '1' corresponds to Label (1 - positive and 0 - negative)
testemailstext = testemails['Email Text'].values.tolist()
testemailstype = testemails['Email Type'].values.tolist()

In [39]:
traintext=[]
for i in range(0, len(trainemailstext)):
    punctuation = []
    for char in str(trainemailstext[i]):
        if char in string.punctuation:
            punctuation.append(char)
    review = ' '.join(punctuation)
    traintext.append(review)
print(len(traintext))

14920


In [14]:
testtext=[]
for i in range(0, len(testemailstext)):
    punctuation = []
    for char in str(testemailstext[i]):
        if char in string.punctuation:
            punctuation.append(char)
    review = ' '.join(punctuation)
    testtext.append(review)

In [19]:
print(len(string.punctuation))

32


In [40]:
train_feature_matrix = []
for text in traintext:
    counts = [0 for i in range(len(string.punctuation))]
    for char in text:
        for i in range(len(string.punctuation)):
            if char == string.punctuation[i]:
                counts[i] += 1
    train_feature_matrix.append(counts)
train_feature_matrix = np.array(train_feature_matrix)
print(len(train_feature_matrix))

14920


In [32]:
test_feature_matrix = []
for text in testtext:
    counts = [0 for i in range(len(string.punctuation))]
    for char in text:
        for i in range(len(string.punctuation)):
            if char == string.punctuation[i]:
                counts[i] += 1
    test_feature_matrix.append(counts)
test_feature_matrix = np.array(test_feature_matrix)

In [42]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(train_feature_matrix, trainemailstype)

MultinomialNB()

In [47]:
y_pred = naive_bayes_classifier.predict(test_feature_matrix)
print(y_pred)

['Safe Email' 'Phishing Email' 'Phishing Email' ... 'Phishing Email'
 'Safe Email' 'Phishing Email']


In [46]:
print(metrics.classification_report(testemailstype, y_pred, target_names=['Positive', 'Negative']))

              precision    recall  f1-score   support

    Positive       0.73      0.62      0.67      1444
    Negative       0.78      0.86      0.82      2286

    accuracy                           0.77      3730
   macro avg       0.76      0.74      0.74      3730
weighted avg       0.76      0.77      0.76      3730



In [45]:
print("Confusion matrix:")
print(metrics.confusion_matrix(testemailstype, y_pred))

Confusion matrix:
[[ 890  554]
 [ 322 1964]]


In [50]:

results = [0,0,0,0]
for i in range(len(testemailstype)):
    if testemailstype[i] == y_pred[i] and testemailstype[i] == 'Phishing Email':
        results[0] += 1 #true positive
    elif testemailstype[i] == y_pred[i] and testemailstype[i] == 'Safe Email':
        results[1] += 1 #true negative
    elif testemailstype[i] != y_pred[i] and testemailstype[i] == 'Phishing Email':
        results[2] += 1 #false negative
    else:
        results[3] += 1 #false positive
print(results)

for total in results:
    print(total/len(testemailstype) * 100)
    

    

[890, 1964, 554, 322]
23.86058981233244
52.65415549597855
14.85254691689008
8.632707774798927


## of 3730 test values:
23.86058981233244% true positive\
52.65415549597855% true negative\
14.85254691689008% false negative\
8.632707774798927% false positive\
76.51474531% correct detection

In [ ]:
oneOff = ['hi michael, please find enclosed vendor banking instructions for a payment that was suppose to go out in the previous week. i need you to process it immediately. i am a bit busy now but will give you a call within the hour regarding the payment. regards, sent from my mobile',
        'dear michael, phishing attacks are a growing threat to online security, but there are steps you can take to protect yourself. here are some best practices to help you stay safe online: keep your software up to date: make sure that your operating system, antivirus software, and other applications are always up to date to reduce your vulnerability to malware. use strong passwords: use strong, unique passwords for each of your online accounts and consider using a password manager to keep them secure. think before you click: always be wary of emails, links and attachments from unknown or suspicious sources. if in doubt, do not click. be cautious of public wi-fi: public wi-fi networks are often secured, so avoid accessing sensitve information when using them. remember, the best defense against phishing attacks is awareness and education. stay vigilant and stay safe online. best regards, hung']

oneOff_feature_matrix = []
for text in oneOff:
    counts = [0 for i in range(len(string.punctuation))]
    for char in text:
        for i in range(len(string.punctuation)):
            if char == string.punctuation[i]:
                counts[i] += 1
    oneOff_feature_matrix.append(counts)
oneOff_feature_matrix = np.array(oneOff_feature_matrix)
print(len(oneOff_feature_matrix))

In [ ]:
oneOff_Y = naive_bayes_classifier.predict(oneOff_feature_matrix)
print(oneOff_Y)